In [1]:
import time
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import random
from functools import reduce
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import oauth2

In [2]:
cid = 'bb34b741629d4173bf4bffa6417a2270'
secret = 'c0c94901909e42e483aac0680375c441'
redirect_uri='http://localhost:7777/callback'
username = 'y1spdovdjdvlieulko3su8e2f'

In [3]:
scope = 'user-top-read playlist-modify-private playlist-modify-public'
token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [4]:
def fetch_audio_features(sp, df):
    playlist = df[['track_id','track_name']] 
    index = 0
    audio_features = []
    
    while index < playlist.shape[0]:
        audio_features += sp.audio_features(playlist.iloc[index:index + 50, 0])
        index += 50
    
    features_list = []
    for features in audio_features:
        features_list.append([features['danceability'],
                              features['acousticness'],
                              features['energy'], 
                              features['tempo'],
                              features['instrumentalness'], 
                              features['loudness'],
                              features['liveness'],
                              features['duration_ms'],
                              features['key'],
                              features['valence'],
                              features['speechiness'],
                              features['mode']
                             ])
    
    df_audio_features = pd.DataFrame(features_list, columns=['danceability', 'acousticness', 'energy','tempo', 
                                                             'instrumentalness', 'loudness', 'liveness','duration_ms', 'key',
                                                             'valence', 'speechiness', 'mode'])
    
    df_playlist_audio_features = pd.concat([playlist, df_audio_features], axis=1)
    df_playlist_audio_features.set_index('track_name', inplace=True, drop=True)
    return df_playlist_audio_features

### Getting the songs from all of Spotify's playlists

In [5]:
# # Getting playlist IDs from each of Spotify's playlists
# playlists = sp.user_playlists('spotify')
# spotify_playlist_ids = []
# while playlists:
#     for i, playlist in enumerate(playlists['items']):
#         spotify_playlist_ids.append(playlist['uri'][-22:])
#     if playlists['next']:
#         playlists = sp.next(playlists)
#     else:
#         playlists = None
# spotify_playlist_ids[:20]

In [6]:
# len(spotify_playlist_ids)

### Getting tracks from Spotify playlists

In [7]:
def getTrackIDs(playlist_id):
    playlist = sp.user_playlist('spotify', playlist_id)
    for item in playlist['tracks']['items'][:50]:
        track = item['track']
        ids.append(track['id'])
    return

In [8]:
# Creating a function get features of each track from track id
def getTrackFeatures(track_id):
  meta = sp.track(track_id)
  features = sp.audio_features(track_id)

  # meta
  track_id = track_id
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  release_date = meta['album']['release_date']
  length = meta['duration_ms']
  popularity = meta['popularity']

  # features
  acousticness = features[0]['acousticness']
  danceability = features[0]['danceability']
  energy = features[0]['energy']
  instrumentalness = features[0]['instrumentalness']
  liveness = features[0]['liveness']
  loudness = features[0]['loudness']
  speechiness = features[0]['speechiness']
  tempo = features[0]['tempo']
  time_signature = features[0]['time_signature']

  track = [track_id, name, album, artist, release_date, length, popularity, danceability, acousticness, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
  return track

The cell below takes about five minutes to run.

In [9]:
# %%time
# # Gathering track ids
# ids = []
# for x in spotify_playlist_ids[:200]:
#     getTrackIDs(x)
# ids[:5]

The cell below takes about 30 minutes to run.

In [10]:
# %%time
# # loop over track ids to get audio features for each track
# tracks = []
# for i in range(len(ids)):
#     try:  
#         track = getTrackFeatures(ids[i])
#         tracks.append(track)
#     except:
#         pass

# # create dataset
# df = pd.DataFrame(tracks, columns = ['track_id', 'name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])
# df.head()

In [11]:
# df.to_csv('playlist_songs.csv',index=False)

In [12]:
df = pd.read_csv('data/playlist_songs.csv')
df.head()

,track_id,name,album,artist,release_date,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,7MAibcTli4IisCtbHKrGMh,Leave The Door Open,Leave The Door Open,Bruno Mars,2021-03-05,242096,90,0.586,0.1820,0.616,0.000000,0.0927,-7.964,0.0324,148.088,4
1,5QO79kh1waicV47BqGRL3g,Save Your Tears,After Hours,The Weeknd,2020-03-20,215626,97,0.680,0.0212,0.826,0.000012,0.5430,-5.487,0.0309,118.051,4
2,1diS6nkxMQc3wwC4G1j0bh,We're Good,Future Nostalgia (The Moonlight Edition),Dua Lipa,2021-02-11,165506,88,0.722,0.0319,0.588,0.000000,0.1830,-5.932,0.0544,134.010,4
3,4u4NyuceXP7Uzh7XFJKCr1,Hold On,Hold On,Justin Bieber,2021-03-05,170813,89,0.658,0.0106,0.634,0.000000,0.1320,-5.797,0.0413,139.980,4
4,3Ofmpyhv5UAQ70mENzB277,Astronaut In The Ocean,Astronaut In The Ocean,Masked Wolf,2021-01-06,132780,94,0.778,0.1750,0.695,0.000000,0.1500,-6.865,0.0913,149.996,4


In [13]:
# Dropping columns that could lead to data leakage
df = df.drop(columns=['name', 'album', 'artist', 'release_date'])
df.head()

,track_id,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,7MAibcTli4IisCtbHKrGMh,242096,90,0.586,0.1820,0.616,0.000000,0.0927,-7.964,0.0324,148.088,4
1,5QO79kh1waicV47BqGRL3g,215626,97,0.680,0.0212,0.826,0.000012,0.5430,-5.487,0.0309,118.051,4
2,1diS6nkxMQc3wwC4G1j0bh,165506,88,0.722,0.0319,0.588,0.000000,0.1830,-5.932,0.0544,134.010,4
3,4u4NyuceXP7Uzh7XFJKCr1,170813,89,0.658,0.0106,0.634,0.000000,0.1320,-5.797,0.0413,139.980,4
4,3Ofmpyhv5UAQ70mENzB277,132780,94,0.778,0.1750,0.695,0.000000,0.1500,-6.865,0.0913,149.996,4


In [14]:
# Dropping duplicated songs
df = df.drop_duplicates(subset=['track_id'])
df['track_id'].value_counts()

5OBTM8RepTDhSKKhDykSPZ    1
6Mfk6R4d6i7dsMF2UGnEfD    1
3dbQ6EkRqF9QybiLBK6OtZ    1
4FQg9eezgtA5DVaYcyCyFi    1
2rf23PCJH09ZAc8eq4yL1B    1
                         ..
4IFh1uImEp02P7wQerkQ9y    1
1h27BIK3pnULp2uH7rSAEd    1
5DwMTTivMOVVAiZxHomSTA    1
2cN3DKBEAq7MAc6aUElXVJ    1
2UJQmwF0yShE7pHcjSzX4X    1
Name: track_id, Length: 8883, dtype: int64

## Getting user's favorite tracks

In [15]:
# Getting top 50 tracks from user
results = sp.current_user_top_tracks(limit=1000, offset=0,time_range='short_term')

In [16]:
# Convert it to Dataframe
track_name = []
track_id = []
artist = []
album = []
duration = []
popularity = []
for i, items in enumerate(results['items']):
        track_name.append(items['name'])
        track_id.append(items['id'])
        artist.append(items["artists"][0]["name"])
        duration.append(items["duration_ms"])
        album.append(items["album"]["name"])
        popularity.append(items["popularity"])

# Create the final df   
df_favourite = pd.DataFrame({ "track_name": track_name, 
                             "album": album, 
                             "track_id": track_id,
                             "artist": artist, 
                             "duration": duration, 
                             "popularity": popularity})

df_favourite.head()

,track_name,album,track_id,artist,duration,popularity
0,Chlorine (Mexico City),Chlorine (Mexico City),4hhc1rMxhTFbTRf9gCFgyR,Twenty One Pilots,238866,44
1,Nee Prashnalu,Kothabangarulokam,0II8bNccmnWMTkc0LHuROD,S. P. Balasubrahmanyam,265168,48
2,Mawa Bro,"Mawa Bro (From ""Das Ka Dhamki"")",3n0944L23I5MwduK8tzZ1C,Ram Miriyala,219428,56
3,Aagi Aagi,Ee Nagaraniki Emaindi,4jYX71osH1K8LA6r3Z7R2d,"Anurag Kulkarni, Manisha Eerabathini",408000,45
4,Maarey Kalaley,Ee Nagaraniki Emaindi,2j182eEbBTo7a8VwrR2IZg,Suraj Santosh,242500,24


In [17]:
%%time
# Getting track features for each song in favorite song dataframe
fav_tracks = []
for track in df_favourite['track_id']:
    try:  
        track = getTrackFeatures(track)
        fav_tracks.append(track)
    except:
        pass

Wall time: 15 s


In [18]:
df_fav = pd.DataFrame(fav_tracks, columns = ['track_id', 'name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])
df_fav.head()

,track_id,name,album,artist,release_date,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,4hhc1rMxhTFbTRf9gCFgyR,Chlorine (Mexico City),Chlorine (Mexico City),Twenty One Pilots,2019-06-21,238866,44,0.773,0.7090,0.188,0.000002,0.103,-12.383,0.0624,79.042,4
1,0II8bNccmnWMTkc0LHuROD,Nee Prashnalu,Kothabangarulokam,Mickey J. Meyer,2008-08-30,265168,48,0.558,0.0953,0.483,0.000000,0.213,-10.749,0.0860,87.884,4
2,3n0944L23I5MwduK8tzZ1C,Mawa Bro,"Mawa Bro (From ""Das Ka Dhamki"")",Ram Miriyala,2023-01-20,219428,56,0.698,0.4630,0.935,0.000107,0.110,-5.911,0.0709,104.992,4
3,4jYX71osH1K8LA6r3Z7R2d,Aagi Aagi,Ee Nagaraniki Emaindi,Vivek Sagar,2018-06-14,408000,45,0.587,0.5740,0.411,0.000018,0.121,-8.245,0.0336,133.066,4
4,2j182eEbBTo7a8VwrR2IZg,Maarey Kalaley,Ee Nagaraniki Emaindi,Vivek Sagar,2018-06-14,242500,24,0.450,0.0230,0.695,0.000202,0.111,-5.790,0.0345,123.928,5


In [19]:
# Dropping columns that could lead to data leakage
df_fav = df_fav.drop(columns=['name', 'album', 'artist', 'release_date'])
df_fav.head()

,track_id,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,4hhc1rMxhTFbTRf9gCFgyR,238866,44,0.773,0.7090,0.188,0.000002,0.103,-12.383,0.0624,79.042,4
1,0II8bNccmnWMTkc0LHuROD,265168,48,0.558,0.0953,0.483,0.000000,0.213,-10.749,0.0860,87.884,4
2,3n0944L23I5MwduK8tzZ1C,219428,56,0.698,0.4630,0.935,0.000107,0.110,-5.911,0.0709,104.992,4
3,4jYX71osH1K8LA6r3Z7R2d,408000,45,0.587,0.5740,0.411,0.000018,0.121,-8.245,0.0336,133.066,4
4,2j182eEbBTo7a8VwrR2IZg,242500,24,0.450,0.0230,0.695,0.000202,0.111,-5.790,0.0345,123.928,5


In [20]:
# Checking for duplicates in df_fav
df_fav['track_id'].value_counts()

1lrgvbZzsjatMee0Uk8o0P    1
1hlFygofCSO8PfIrAougLY    1
2JzZzZUQj3Qff7wapcbKjc    1
0TQBZ1MC2Y6CuDjvMCFHMg    1
1jaPWaMCgXLdUEEiEaKy6I    1
7JJmb5XwzOO8jgpou264Ml    1
51cJFmn6j0LUaxef2lQ3dz    1
2ptZ5FOsn59LUCPfYlOoeA    1
5haAVKCKAfb7IYLplcao0h    1
0VjIjW4GlUZAMYd2vXMi3b    1
5tvbBnrlNvysgVyepyPr0M    1
4PhsKqMdgMEUSstTDAmMpg    1
4ELXnfY3YMJi47WfcRl5W1    1
376Gg7Oxo5C5jUWDWpVXeI    1
0II8bNccmnWMTkc0LHuROD    1
2m89IdT7sJdSky0Fymg7r2    1
57wEDajjYl5o4yEO1udmP4    1
2niTwfsUYC0sfrUMySeqdK    1
0VO8gYVDSwM1Qdd2GsMoYK    1
4I4LQkTBE26JJpGsjsIHtX    1
3n0944L23I5MwduK8tzZ1C    1
5QO79kh1waicV47BqGRL3g    1
7L2D6W7e8mn0zf8cH78Ch4    1
59eeV0SIeyd431uyjAWCRe    1
2LBqCSwhJGcFQeTHMVGwy3    1
5VxL1b7OOpc1m0n2eAiAoo    1
6LsAAHotRLMOHfCsSfYCsz    1
2GyA33q5rti5IxkMQemRDH    1
473dofPPXF0nFDMq1RAQc8    1
7hMKAmUlUUYa5qEiHWK4bk    1
7t8gUSPNVJm0bRMx4BKApC    1
09mEdoA6zrmBPgTEN5qXmN    1
2p8IUWQDrpjuFltbdgLOag    1
3RiPr603aXAoi4GHyXx0uy    1
2o5YDkVrId7JeumduZJ6NL    1
4ewe55AoVtNZQZMihVmp

In [21]:
# Creating favorite column to use in classification
df_fav['favorite'] = 1
df['favorite'] = 0 

In [22]:
# Checking if both datasets have the same columns
df.columns == df_fav.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

## Preparing dataset for model creation

In [23]:
print(df.shape)
print(df_fav.shape)

(8883, 13)
(50, 13)


In [24]:
# Combining the favorite dataframe with the dataframe of Spotify's songs
combined = pd.concat([df, df_fav])
combined.shape

(8933, 13)

In [25]:
combined.favorite.value_counts()

0    8883
1      50
Name: favorite, dtype: int64

The favorite songs to not favorite songs ratio is imbalanced, I will need to fix this when building the model

In [26]:
# Creating dataframe of favorite songs
df_fav = combined.loc[combined['favorite'] == 1]
df_fav.head()

,track_id,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,favorite
0,4hhc1rMxhTFbTRf9gCFgyR,238866,44,0.773,0.7090,0.188,0.000002,0.103,-12.383,0.0624,79.042,4,1
1,0II8bNccmnWMTkc0LHuROD,265168,48,0.558,0.0953,0.483,0.000000,0.213,-10.749,0.0860,87.884,4,1
2,3n0944L23I5MwduK8tzZ1C,219428,56,0.698,0.4630,0.935,0.000107,0.110,-5.911,0.0709,104.992,4,1
3,4jYX71osH1K8LA6r3Z7R2d,408000,45,0.587,0.5740,0.411,0.000018,0.121,-8.245,0.0336,133.066,4,1
4,2j182eEbBTo7a8VwrR2IZg,242500,24,0.450,0.0230,0.695,0.000202,0.111,-5.790,0.0345,123.928,5,1


In [27]:
# Removing favorite songs from playlist songs
df = combined.loc[combined['favorite'] != 1]
df.shape

(8883, 13)

In [28]:
df_fav.shape

(50, 13)

In [29]:
# Saving these dataframes to use in model creation
df.to_csv('encoded_playlist_songs.csv', index=False)
df_fav.to_csv('favorite_songs.csv', index=False)